# Simple Model Management; Dimensionality Reduction and Regressions

## Preparing this Workspace

For this tutorial, please set up a directory on TALC JupyterHub and upload the following files (all available on D2L) into it:

* `DimensionalityAndRegression.ipynb`
* `diabetes.csv`
* `iris.csv`

## SciKit-Learn API Style

For this tutorial, we will use the utilities provided by SciKit-learn. For the sake of time, we will only be touching on the simpler, common utilities provided by this package; links to more in-depth discussion of the full utilities available will be linked where applicable.

For nearly all utilities in SciKit-Learn, we must first declare an *instance* of the utility. This instance is what we will actually use to analyze and/or transform the data, and how we declare it will determine how it behaves. This declaration is a function, taking a number of parameters which are stored within the instance and used when it processes the data its given. For example, the `SimpleImputer` utility (which replicates the imputation functionality we discussed last week) can take a `strategy` argument, which specifies how it should fill in non-existant values. We will discuss these arguments as they arise.

Once declared, almost all SciKit-Learn utilities follow a common pattern in how they are used. This is accomplished via the use of shared functions which all utilities share, allowing for most forms of analysis post-utility initialization to be very straightforward:

* `fit`: Requests that the utility "analyzes" the provided dataset, in preparation for its application. This can be as simple as checking against a condition, or as advanced as learning complex relations within a set of features in labels. This returns a (now updated) version of the utility that called it.
* `transform`: Requests that the utility now applies what it has learned to the provided dataset. This dataset can be the one it originally analyzed, but does not have to be! It can instead be another dataset that shares the same features as the one it originally learned from. This returns the results of this application, as a modified copy of the original dataset; *the original dataset is not modified in this process*
* `fit_transform`: Performs the `fit` and `transform` functions above, in that order, on the same dataset. Returns the resulting (modified) dataset. If the utility lacks a `transform` function, it will also lack this function.
* `predict`: Like `transform` before it, this requests that the utility take the provided data and predict the correct target labels for them. This dataset does not need to be the one it originally learned from; it can be a different dataset that shares the same features instead. The predicted target labels are returned to you, for further analysis. All utilities will have `transform` or `predict`, but not both.
* `score`: Evaluates how effectively the utility performs, usually with a higher value being better, when given a set of input features and their "true" label values. The metric for this evaluation differs depending on the utility, though this is usually either a form of accuracy or measure of error rate. Note that most utilities that do not "learn" parameters will not have this function.

## What is Data Dimensionality?

Values within a dataset, much like co-ordinates on a piece of paper, are often used to represent a "position" of entries within the full range of available data. As such, features within a dataset are often reffered to interchangeably as a "dimension" within a dataset, resulting in the number of features a dataset has being used to describe how many dimensions it has. Approaching data this way allows us to better grasp how data can be worked with, and how we can transform it to better suit our needs.

However, just like a 20 dimensional plot is near impossible to understand and interpret, data with too many dimensions can be come unweildy to work with and lead to confusion. Due to how machine learning systems are designed, they are prone to a number of errors when fed with high dimensional data:

1. Assuming all dimensions are equally worth using, even if some are completely irrelevant to the task at hand (similar to a "red herring" in a mystery novel; this information was given to me for a reason, right?)
2. Treating all dimensions are effectively unique, even though multiple dimensions in the dataset may functionally represent the same thing (i.e. hours of daylight and solar power production likely both represent the amount of light in an environment)
3. Accidentally implying relations where there are none (ice cream sales and murder rate are correlated to one another; in reality, they just both happen to increase in the summer)

To combat these errors, a number of "dimensionality reduction" techniques have been devised. Thankfully, these problems are common enough that utilities to automate these processes have been coded for us.

For this tutorial, we will be using modified version of two datasets; the [iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) dataset (which provides us with a species classification task), and the [diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes) dataset (which provides us with a disease progression task instead). Note that the latter has had its data normalized, resulting the seemingly nonsensical values present within some of its columns. Both have also been scrambled, and had a new "id" metadata column added to them for identification purpouses.

In [1]:
import numpy as np
import pandas as pd

# Load our iris data
iris_df = pd.read_csv("iris.csv")

# Load our diabetes data
diab_df = pd.read_csv("diabetes.csv")

In [2]:
iris_df

,id,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,3,6.7,3.1,4.7,1.5,1
1,13,5.7,2.8,4.1,1.3,1
2,20,6.1,3.0,4.6,1.4,1
3,24,6.7,3.3,5.7,2.1,2
4,29,5.4,3.0,4.5,1.5,1
...,...,...,...,...,...,...
145,843,4.6,3.4,1.4,0.3,0
146,851,6.4,2.7,5.3,1.9,2
147,857,5.1,3.8,1.9,0.4,0
148,865,5.7,3.8,1.7,0.3,0


In [3]:
diab_df

,id,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,response of interest
0,8,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,12,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,21,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,29,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,35,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...,...
437,2432,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,2433,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,2437,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,2440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


Before doing any analysis which involves feature modification, we generally want to split the dataset's columns three ways:

* The metadata columns (which label the information for the purpouses of its management, but have little to no signficance in how we actually interpret the data)
* The feature columns (which do hold relevant information on how we interpret the dataset, informing any analyses)
* The label/target columns (which are the column(s) we intend to design our analyses around interpretting)

Examples of how you might want to do this is shown below:

In [4]:
# Iris data preparation w/ labels (loc)
iris_meta = iris_df.loc[:, 'id']
iris_features = iris_df.loc[:, 'sepal length (cm)':'petal width (cm)']
iris_target = iris_df.loc[:, 'species']

In [5]:
# Diabetes data preparation w/ positions (iloc)
diab_meta = diab_df.iloc[:, 0]
diab_features = diab_df.iloc[:, 1:11]
diab_target = diab_df.iloc[:, 11]

## Dimensionality Reduction Techniques

### Option 1; Feature Selection

This simplest way to reduce the number of dimensions in a dataset is as straightforward as can be; just start throwing away stuff we don't want! Depending on your circumstances, we can accomplish this one of two ways (though these can also be combined if needed):

1. Unsupervised feature selection (select features based on metrics implicit to the features themselves)
2. Supervised feature selection (select features based on a metric that represents how we would expect them to impact the final target model)

SciKit-Learn has a well developed suite of utilities to handle both of these circumstances; for this tutorial, we will only touch on the simple methods they provide. An in-depth discussion of the full utilities available can be accessed [here](https://scikit-learn.org/stable/modules/feature_selection.html)

#### 1. Unsupervised Example; Selection by Variance

A simple unsupervised option is to remove features with low *variance*. This is based on the idea that features that have low variation are likely only represent error and/or uninformative noise. For this, we can use the `VarianceThreshold` utility to automate the process. To instantiate it, one parameter is needed:

* `threshold`: The minimum variance that is required for a feature to be conserved. Remember variance is the square of standard deviation (which measures the average distance from the mean), so this value may need to be square-rooted to acomplish your intended goal. This value will also scale depending on the average scale of the features being used, with ).

We will apply this to the iris dataset, using `0.25` as our variance (chosen as the data is measured in centimeters, and samples differing by less than 0.5cm on average are likely the result of random genetic differences rather than species specific ones)

In [6]:
from sklearn.feature_selection import VarianceThreshold

feature_selector = VarianceThreshold(threshold=0.25)

Once this is completed, we can apply it to our datasets using the shared functions mentioned prior. Note that this returns a multi-dimensional array, *not* a DataFrame, so we need to convert it back:

In [7]:
# Fit and transform the data
feature_selector.fit(iris_features)
trimmed_iris_features = feature_selector.transform(iris_features)

# Rebuild a DataFrame from the results
feature_names = feature_selector.get_feature_names_out()
trimmed_iris_features = pd.DataFrame(data=trimmed_iris_features, columns=feature_names)

trimmed_iris_features

,sepal length (cm),petal length (cm),petal width (cm)
0,6.7,4.7,1.5
1,5.7,4.1,1.3
2,6.1,4.6,1.4
3,6.7,5.7,2.1
4,5.4,4.5,1.5
...,...,...,...
145,4.6,1.4,0.3
146,6.4,5.3,1.9
147,5.1,1.9,0.4
148,5.7,1.7,0.3


Nice; only one feature of the iris dataset was had too little variation, resulting in 3 of our 4 features being maintaned.

#### 2. Supervised Example; `f_regression` guided K-Best

Alternatively, we can select a feature based on how "good" it at aiding our prediction task. Most methods in SciKit-Learn allow us to define how this is measured, as needed; depending on whether our target is categorical or continuous, a number of functions exist to do so:

* For continuous: `f_regression` (F-Test on continous targets) and `mutual_info_regression` (measure of how dependent the target is on input feature values)
* For categorical: `chi2` (chi-squared test), `f_classif` (F-Test on categorical targets), and `mutual_info_classif` (measure of how dependent the target is on input feature class)

Similarly, we have options for how we want to determine the cutoff for "goodness" of our features:

* `SelectKBest`: Select the an explicit number (`k`) of the best features in the dataset
* `SelectPercentile`: Select best percentage of features in the dataset, rounded to the nearest whole number.

Both utilities requires two arguments when declared

* `score_func`: The scoring function to apply to the method; these methods assume higher values are preferred to lower ones!
* `k`/`percentile`: The amount of features to preserve (respectively as a whole number or decimal percentile for `KBest` and `Percentile` respectively)

These features, being supervised, also need our target metric to be provided to them as a second parameter when the `fit` command is called. Below is an example of this using our diabetes dataset, with `SelectKBest` with an `f_regression` scoring system and preserving the top 5 features:

In [8]:
# Import the feature selection utility
from sklearn.feature_selection import SelectKBest
# Import the scoring function for it to use
from sklearn.feature_selection import f_regression

# Initialize the utility
feature_selector = SelectKBest(score_func=f_regression, k=5)

# Fit and transform our diabetes dataset
feature_selector.fit(diab_features, diab_target)
trimmed_diab_features = feature_selector.transform(diab_features)

# Rebuild a DataFrame from the results
feature_names = feature_selector.get_feature_names_out()
trimmed_diab_features = pd.DataFrame(data=trimmed_diab_features, columns=feature_names)

trimmed_diab_features

,bmi,bp,s3,s4,s5
0,0.061696,0.021872,-0.043401,-0.002592,0.019908
1,-0.051474,-0.026328,0.074412,-0.039493,-0.068330
2,0.044451,-0.005671,-0.032356,-0.002592,0.002864
3,-0.011595,-0.036656,-0.036038,0.034309,0.022692
4,-0.036385,0.021872,0.008142,-0.002592,-0.031991
...,...,...,...,...,...
437,0.019662,0.059744,-0.028674,-0.002592,0.031193
438,-0.015906,-0.067642,-0.028674,0.034309,-0.018118
439,-0.015906,0.017282,-0.024993,-0.011080,-0.046879
440,0.039062,0.001215,-0.028674,0.026560,0.044528


As requested, only 5 features remain, selected to be the best performing based on a continuous F-test.

<img style="float: left; padding:10px;" src="https://img.icons8.com/office/344/faq.png" width=80 height=80 />

**Consider the Following**

Which of the original three issues mentioned prior does this method solve? 

Are their potential problems that doing dimensionality reduction this way may introduce?

### Option 2; Feature Extraction

Instead of deleting features entirely, we can transform them into new, more condensed features instead. Like before, we can do this two ways:

* Unsupervised; try to preserve as much variation as possible.
* Supervised; try to preserve as much *relevant* variation as possible, based on how they appear to relate to the target feature(s).

While alternatives exist, by far and away the most common approaches for each of these is **Principal Component Analysis (PCA)** and **Linear Discriminant Analysis (LDA)**, respectively

#### 1. Unsupervised Example; PCA

To quickly review, PCA attempts to transform the provided features into *components*, each representing as much variation as possible within the full dataset. This means the first component is produces represents the most variation, followed by the second, and so on. As it measures variation, you should aim to have each feature share the same approximate scale; otherwise features which happen to be scaled lower than others (by centimeters rather than meters, for example) may be ignored by PCA simply because their variance is numerically lower. In SciKit-Learn, we can run this transformation using the `PCA` utility. When we initialize this utility, you may specify the following argument:

* `n_components`: The number of Principal Components that should be kept. If not specified, will default to either the number of features you provided minus 1, or the number of samples you provided, whichever is smaller.

An example of this process is shown below, reducing the diabetes dataset down to 5 features again:

In [9]:
from sklearn.decomposition import PCA

# Transforming our data
pca = PCA(n_components=5)
pca_diab_features = pca.fit_transform(diab_features)

# Restoring it to dataframe format
col_names = []
for i in range(5):
    col_names.append(f"pc{i}")

pca_diab_features = pd.DataFrame(data=pca_diab_features, columns=col_names)

pca_diab_features

,pc0,pc1,pc2,pc3,pc4
0,0.027931,-0.092601,0.028027,-0.003939,-0.012207
1,-0.134686,0.065263,0.001328,-0.022356,-0.006813
2,0.012945,-0.077764,0.035164,-0.037647,-0.055357
3,0.002345,0.018182,-0.095750,0.065318,0.012154
4,-0.035981,0.038621,-0.002724,-0.006541,-0.006343
...,...,...,...,...,...
437,0.058958,-0.049275,0.044173,-0.031215,0.009718
438,0.060155,0.036211,-0.083249,-0.053914,-0.004472
439,-0.009763,-0.057337,0.023596,-0.064372,-0.006739
440,0.032956,0.009994,-0.041321,0.076903,0.005691


Note that we can also inspect the PCA utility itself to see how much variation each component "explains" from the original feature set by querying it's `explained_variance_ratio_`:

In [10]:
pca.explained_variance_ratio_

array([0.40242142, 0.14923182, 0.12059623, 0.09554764, 0.06621856])

The above tells us the first principal component represents ~40.24% of the original datasets variation, the second ~14.92%, and so on. The total variance represented can be calculated simply by taking the sum of this list:

In [11]:
np.sum(pca.explained_variance_ratio_)

0.8340156689459763

83.4% of the dataset's variation in 50% of the features; not to shabby!

#### 2: Supervised Example; LDA 

Similar to PCA prior, LDA aims to reduce the number of features by transforming them. However, unlike PCA, the method of doing so aims to preserve only informative variation, based on a set of provided target labels. Despite this change, LDA requires similar considerations for the data; namely, since it is still aiming to preserve variation, standardizing the scale of each feature is important to prevent issues from cropping up. In SciKit-Learn, this transformation is handled by the `LinearDiscriminantAnalysis` utility (`LDA` was already taken up by another utility, hence no acronym). We can get around this using the same technique of renaming `pandas` to `pd`, however:

In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

Like PCA, you can specify the following argument to tailor its processing to your needs:

* `n_components`: The number of Linear Discriminants that should be kept. If not specified, will default to either the number of features you provided minus 1, or the number of samples you provided, whichever is smaller.

Below is the PCA analysis done prior, replicated with LDA:

In [13]:
# Transforming our data
lda = LDA(n_components=5)
lda_diab_features = lda.fit_transform(diab_features, diab_target)

# Restoring it to dataframe format
col_names = []
for i in range(5):
    col_names.append(f"ld{i}")

lda_diab_features = pd.DataFrame(data=lda_diab_features, columns=col_names)

lda_diab_features

,ld0,ld1,ld2,ld3,ld4
0,1.663158,-1.367074,-0.715150,0.001265,-0.300979
1,-2.312019,0.724032,-1.024317,-1.088293,-1.718292
2,0.750935,-1.952337,-0.414892,-0.697182,-0.940487
3,0.425156,0.924826,1.281073,0.399299,1.288591
4,-0.760852,0.154349,-1.057222,-1.435059,0.379962
...,...,...,...,...,...
437,1.253643,-1.214335,-1.384590,0.339403,0.592753
438,-1.514726,-1.279373,2.298311,0.521716,0.622083
439,-0.810260,-1.568301,-0.436171,-0.532425,0.469731
440,1.684606,0.659046,0.177554,0.065803,0.280087


Identically to PCA, the explained variance of the Linear Discriminants can also be analyzed:

In [14]:
lda.explained_variance_ratio_

array([0.2520478 , 0.11323441, 0.11121644, 0.09763492, 0.09338717])

In [15]:
np.sum(lda.explained_variance_ratio_)

0.6675207483488869

Notice that this has dropped compared to PCA; while we are still improving on the variance-to-features ratio (66.8% for 50% features of not bad at all), the restriction that only *relevant* variation is preserved reduces the amount of overall variation conserved in the analysis.

<img style="float: left; padding:10px;" src="https://img.icons8.com/office/344/faq.png" width=80 height=80 />

**Consider the Following**

Which of the original three issues mentioned prior do these methods solve? 

Are their potential problems that doing dimensionality reduction using these methods may introduce?

## Regression

Now that our data is finally prepared, we can move to actually interpret it. Regression is one of the simplest methods for doing so, as it only attempts to identify "relations" btween features and the target, without considering any other factors. Depending on the type of data we are working on, one of two common forms of regression exist:

* `LinearRegression`; for target metrics which are continuous
* `LogisticRegression`; for target metrics which are categorical

Each of these do not need any arguments to initialize, but the following can be specified if you would like to tune them.

* `n_jobs`: Defaults to `1`, but can be set higher if you are running on a machine that has mutliple available CPU cores that can be run in parralel. Can be set to `-1` to use all available cores if you are not sure.

### Logistic Regression Example

Lets first run our regression on the trimmed Iris data to see how it works. Our target is the `species` of the Iris, which is categorical, so we will use `LogisticRegression` here:

In [16]:
from sklearn.linear_model import LogisticRegression

# Initialize the model
logreg = LogisticRegression(n_jobs=-1)

# Fit it to our data
logreg.fit(trimmed_iris_features, iris_target)

LogisticRegression(n_jobs=-1)

Once a model is "trained" like this, it can then be used to predict the target class for data containing the same features (for the sake of simplicity, this is just a subset of the full iris dataset, but any data works):

In [17]:
# Create our subset
iris_feature_subset = trimmed_iris_features.iloc[:20, :]
iris_target_subset = iris_target.iloc[:20] # No column index because there is only one column

# Predict the subset's label using the model we just trained
predicted_labels = logreg.predict(iris_feature_subset)

# Place these predictions in a dataframe alongside the "true" labels for comparison
iris_comp_df = pd.DataFrame(data = [predicted_labels, iris_target_subset], index=['Predicted', 'True']).T # The T is just to make sure the data is orientated correctly

iris_comp_df

,Predicted,True
0,1,1
1,1,1
2,1,1
3,2,2
4,1,1
5,1,1
6,1,2
7,0,0
8,0,0
9,0,0


Not bad! Looking over the data, only one mistake appears to have been made (row 6). We can use the `score` function instead (which just requires the features to build predictions off of, and the true labels to compare said predictions against) to get an numerical representation of this rate of error, should we not want to inspect the data manually:

In [18]:
logreg.score(iris_feature_subset, iris_target_subset)

0.95

In the case of Logistic Regression, this score is reported as accuracy. Thus, our model had a 95% accuracy on our subset; not bad at all!

### Linear Regression Example

As our diabetes data is continuous, we instead need to use `LinearRegression` instead. The workflow is otherwise the same, though the final score is reported a little differently. This is demonstrated below with our LDA-transformed diabetes data:

In [19]:
from sklearn.linear_model import LinearRegression

# Initialize the model
lda_linreg = LinearRegression(n_jobs=-1)

# Fit it to our data
lda_linreg.fit(lda_diab_features, diab_target)

LinearRegression(n_jobs=-1)

In [20]:
# Create our evaluation subset
diab_lda_feature_subset = lda_diab_features.iloc[:30, :]
diab_target_subset = diab_target.iloc[:30] # No column index because there is only one column

# Evaluate the model
lda_linreg.score(diab_lda_feature_subset, diab_target_subset)

0.43223345760777465

For linear regression, the score reported represents "the coefficient of determination" for the prediction. This is a measure of how much a predicted value deviates from the true target, with a value of `1.0` representing perfect prediction, and a score of `0.0` representing the accuracy of a completely neive system (one which just predicts the most common answer, without any further consideration). As such, our score of 0.43, while not ideal, is a good sign; the model is clearly making informed predictions and doing so with reasonable predictive accuracy. Let's check whether using our PCA data improves things:

In [21]:
# Re-Initialize the model
pca_linreg = LinearRegression(n_jobs=-1)

# Fit it to our data
pca_linreg.fit(pca_diab_features, diab_target)

LinearRegression(n_jobs=-1)

In [22]:
# Create our evaluation subset
diab_pca_feature_subset = pca_diab_features.iloc[:30, :]
diab_target_subset = diab_target.iloc[:30] # No column index because there is only one column

# Evaluate the model
pca_linreg.score(diab_pca_feature_subset, diab_target_subset)

0.3398952724319929

Seems that, despite PCA conserving more variation, said variation actually hurts our results in this case, rather than improving them.

<img style="float: left; padding:10px;" src="https://img.icons8.com/office/344/faq.png" width=80 height=80 />

**Consider the Following**

Do these relations represent causal interactions between the features and our target? If not, what do they represent instead?

Are their types of relations which regression may fail to account for?